In [2]:
import pandas as pd
user = pd.read_csv(r'C:\Users\milan\Documents\GitHub\DSC-630-Project\Propensity.csv')
user.head()

C:\Users\milan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (20,21,30) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,application_id,bind_date,open_to_submit_min,submit_to_approve_min,channel_combined,business_line,open_date,bind,pre_approval_interactions,post_approval_pre_bind_interactions,...,travel_countries_count,med_conditions_count,bmi,age,income,annual_premium_estimated,income_comb,income_grp_v1,income_grp_v2,income_prem_ratio
0,cc3090db-2ed8-4fc5-8f66-f3ef8bfef12b,2021-08-01,5,2,Lemonade,protect,2021-07-24,1,0,0,...,NaN,1.0,26.58,57.53,360000.0,1112.04,360000.0,$100k+,over $50k,323.729362
1,9bffeb0e-bf7d-4544-a175-df737563b530,NaN,4,2,Lemonade,protect,2021-08-12,0,0,0,...,NaN,1.0,33.90,41.32,57000.0,NaN,57000.0,$50k-$100k,over $50k,NaN
2,38c5539f-aa55-406f-8356-91859e67502a,NaN,0,2,Lemonade,protect,2021-08-22,0,0,0,...,NaN,1.0,22.96,39.60,84000.0,NaN,84000.0,$50k-$100k,over $50k,NaN
3,ea0c0f26-b08a-4121-92b4-04b36ac16442,2021-08-10,4,2,Lemonade,protect,2021-08-10,1,0,0,...,NaN,1.0,34.17,41.93,245000.0,2541.96,245000.0,$100k+,over $50k,96.382319
4,0a40356d-aaa0-4aaa-9bbd-5f0f0534608c,NaN,9,1,Lemonade,protect,2021-12-04,0,0,0,...,NaN,1.0,37.12,42.82,78500.0,NaN,78500.0,$50k-$100k,over $50k,NaN


In [4]:
user.shape

(127691, 61)

In [3]:
quote = pd.read_csv(r'C:\Users\milan\Documents\GitHub\DSC-630-Project\quote_to_approved.csv')
quote.head()

,created_at,business_line,quote_id,application_id,face_amount,product,quoted_risk_class,approved_risk_class,quoted_premium,approved_premium,premium_difference,premium_pct_diff
0,2019-10-27,direct,47386ebc-4405-40ca-b2c1-79b24a537e97,844a2118-1c9e-4f50-86ed-970937f8ecfc,200000,BT2002,select_tb,select_nt,5.0,33.33,-28.33,-5.666
1,2022-04-11,direct,ee741da6-788e-41cb-8b23-ab23bf07fecc,e3f51c27-eba2-4f36-86d2-f204baf66455,50000,BT1004,elite_nt,elite_nt,7.5,7.50,0.00,0.000
2,2021-10-26,direct,f05908fa-25df-4219-89fb-b30f2bb517a3,8cf1757c-f644-4082-a0bb-8b3ecb50d79b,50000,BT1004,elite_nt,elite_nt,7.5,7.50,0.00,0.000
3,2021-04-28,direct,4e8cf39c-5334-451f-8770-d3de35d1b4ea,42c460b7-3d17-4c27-8f11-535b5da44d2f,50000,BT1004,elite_nt,super_elite_nt,7.5,7.50,0.00,0.000
4,2021-04-19,advisor,e573f1be-9ed2-4ff9-8597-20859417cad0,7145a861-c133-4330-918b-80d4199078ca,50000,BT1004,elite_nt,super_elite_nt,7.5,7.50,0.00,0.000


In [5]:
quote.shape

(101416, 12)

In [7]:
df = pd.merge(user, quote, on = 'application_id')
df.shape

(44844, 72)